# アイテムベースの協調フィルタリング

### 評価値データベース

- データを pandas DataFrame として格納

![24](slides/24-table.png)

- 「アイテム5」に対する嗜好に似たアイテムはどれか？
  - ユーザのアイテムに対する嗜好の類似度を用いて評価を予測
- 下記の評価値データベースでは
  - アイテム5は、アイテム1, 4と似ている（ように見える）

In [ ]:
import numpy as np
import pandas as pd

# リストで元データを作成
data1 = [
    [5, 3, 4, 4],     # You
    [3, 1, 2, 3, 3],  # User1
    [4, 3, 4, 3, 5],  # User2
    [3, 3, 1, 5, 4],  # User3
    [1, 5, 5, 2, 1]   # User4
]

data2 = [
    [np.nan, 3, 4, 4],     # You
    [3, 1, 2, 3, 3],  # User1
    [4, 3, 4, 3, 5],  # User2
    [3, 3, 1, 5, 4],  # User3
    [1, 5, 5, 2, 1]   # User4
]

# pandas DataFrame を上記の data1, data2 から作成
# - columns: 列ラベルの指定
# - index: 行ラベルの指定
db1 = pd.DataFrame(data1,
                   columns=['Item1', 'Item2', 'Item3', 'Item4', 'Item5'],
                   index=['You', 'User1', 'User2', 'User3', 'User4'])

db2 = pd.DataFrame(data2,
                   columns=['Item1', 'Item2', 'Item3', 'Item4', 'Item5'],
                   index=['You', 'User1', 'User2', 'User3', 'User4'])

# 確認
# - 'You', 'Item5' は値が無いため NaN (np.nan) 
# - 'Item5' 列は 'You' が NaN（浮動小数点数）のため、他の値も浮動小数点数
db1

### 類似度を計算する関数の定義

![8](slides/8.png)

In [ ]:
# コサイン類似度（= 1 - コサイン距離）
from scipy.spatial.distance import cosine

def similarity_cosine(df, l1, l2):
    a = df[[l1, l2]].dropna()
    if len(a[l1]) < 2:
        return np.nan
    return 1 - cosine(a[l1], a[l2])

similarity_cosine(db1, 'Item1', 'Item5')
#similarity_cosine(db2, 'Item4', 'Item5')

![9](slides/9.png)

In [ ]:
# 行の平均値を計算し、平均評価値の列を追加
db1['mean'] = db1.mean(axis=1)
# 確認
db1

In [ ]:
# DataFrame の複製を作成
db1_adj = db1.copy()
# 平均評価値を引き調整
for i in db1_adj.columns.drop('mean'):
    db1_adj[i] = db1_adj[i] - db1_adj['mean']
# mean 列を削除
db1_adj = db1_adj.drop('mean', axis=1)
# 確認
db1_adj

![10](slides/10.png)

In [ ]:
# 類似度行を追加し、NaN で初期化
db1_adj.loc['similarity'] = np.nan
# 各アイテムについて類似度を計算し、DataFrameに代入
for i in db1_adj.columns.drop('Item5'):
    db1_adj.at['similarity', i] = similarity_cosine(db1_adj, i, 'Item5')
# 確認
db1_adj

![11](slides/11.png)

In [ ]:
# 正の類似度を持つアイテムのみを取り出す
# - query は行に対してのみ処理可能なため T で転置する
db1_sim = db1_adj.T.query('similarity > 0')
# 確認
db1_sim

In [ ]:
# 加重平均を計算
avg = (db1_sim['You'] * db1_sim.similarity).sum() / db1_sim.similarity.sum()
# 平均評価値に加算
db1.at['You', 'mean'] + avg